Scraper for http://www.piano-e-competition.com/

Inspired by:  
https://github.com/djosix/Performance-RNN-PyTorch/blob/794125a1d7f8fd22a1a976b43f52e1852bb7e346/dataset/scripts/ecomp_piano_downloader.sh

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
import json

In [2]:
import re
from pathlib import Path
from midi_data import save_json, load_json

In [3]:
base_url = 'http://www.piano-e-competition.com'
urls = [
    'http://www.piano-e-competition.com/ecompetition/midi_2002.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2004.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2006.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2008.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2009.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2011.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2013.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2014.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2015.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2017.asp',
    'http://www.piano-e-competition.com/ecompetition/midi_2018.asp'
]

In [4]:
path = Path('data/midi/original/ecomp')
meta_path = path/'song_list.json'

In [5]:
def get_link_data(link):
    artist = str(link.parent.previous.previous)
    title = link.text
    midi_url = base_url + '/' + link.attrs['href']
    return artist, title, midi_url

In [6]:
def download_midi(url, out_file):
    if out_file.exists(): return
    r = requests.get(url)
    open(out_file, 'wb').write(r.content)

In [7]:
def download_links(midi_links, results):
    for link in midi_links:
        artist,title,midi_url = get_link_data(link)
        filename = Path(midi_url).name
        year = Path(url).with_suffix('').name[-4:]
        out_file = path/year/filename
        out_file.parent.mkdir(parents=True, exist_ok=True)
        download_midi(midi_url, out_file)
        results[filename] = { 'artist': artist, 'title': title, 'year': year }

In [8]:
results = {}

In [9]:
for url in urls:
    response_tmp = requests.get(url)
    soup = BeautifulSoup(response_tmp.text, 'html.parser')
    midi_links = soup.find_all('a', {'href': re.compile(r'\.(mid|MID)')})
    print('Found:', len(midi_links))
    download_links(midi_links, results)

Found: 122
Found: 295
Found: 347
Found: 303
Found: 331
Found: 175
Found: 159
Found: 353
Found: 178
Found: 186
Found: 305


In [10]:
save_json(results, meta_path)

In [ ]:
results